In [7]:
import os
print(os.environ['CONDA_DEFAULT_ENV'])

geo_env


In [8]:
import geopandas as gpd
import rasterio
from rasterio.plot import show
from shapely.geometry import shape
from rasterio.features import shapes as shape_rasterio
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [9]:
def read_geotiff(file_path):
    with rasterio.Env():
        with rasterio.open(file_path, 'r', driver='GTiff', sharing=False) as src:
            image = src.read(1)  # Assuming a single-band image
            transform = src.transform

    return image, transform

In [10]:
def kmeans_clustering(image, num_clusters):
    rows, cols = image.shape
    reshaped_img = image.reshape((-1, 1))

    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    labels = kmeans.fit_predict(reshaped_img)

    segmented_labels = labels.reshape((rows, cols))

    return segmented_labels

In [11]:
def create_geodataframe(segmented_labels, transform):
    # Use rasterio.features.shapes to generate shapes from the labels
    shapes = shape_rasterio(segmented_labels.astype('int16'), transform=transform)

    # Extract geometries and values using rasterio.mask
    geometries, values = zip(*[(geom, value) for geom, value in shapes if value != 0])

    # Create a GeoDataFrame with a valid geometry column
    gdf = gpd.GeoDataFrame(geometry=list(geometries), data={'value': list(values)}, crs='EPSG:4326')

    return gdf

In [12]:

geotiff_path = './subset_0_of_S1A_IW_GRDH_1SDV_20240113T092308_20240113T092333_052086_064B95_F370_Cal_Spk_TC.tif'
num_clusters = 2

image, transform = read_geotiff(geotiff_path)
segmented_labels = kmeans_clustering(image, num_clusters)
gdf = create_geodataframe(segmented_labels, transform)

fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, edgecolor='black', facecolor='none')
show(image, ax=ax, cmap='gray')
plt.show()

TypeError: Input must be valid geometry objects: {'type': 'Polygon', 'coordinates': [[(-55.35122307757732, -1.9592725863960416), (-55.35122307757732, -1.9593624179244535), (-55.35095358299209, -1.9593624179244535), (-55.35095358299209, -1.9592725863960416), (-55.35122307757732, -1.9592725863960416)]]}